In [22]:
%load_ext zipline

The zipline extension is already loaded. To reload it, use:
  %reload_ext zipline


In [20]:
%%zipline --start 2015-1-1 --end 2018-1-1  --capital-base 10000 -o ../data/strategy3.pickle

"""Dual Moving Average Crossover algorithm.

This algorithm buys apple once its short moving average crosses
its long moving average (indicating upwards momentum) and sells
its shares once the averages cross again (indicating downwards
momentum).

"""

from zipline.api import order, record, symbol, order_target_percent
from zipline.finance import commission, slippage
# Import exponential moving average from talib wrapper
from talib import EMA


def initialize(context):
    context.asset = symbol('AAPL')

    # To keep track of whether we invested in the stock or not
    context.invested = False
 
    context.set_commission(commission.PerShare(cost=.0075, min_trade_cost=1.0))
    context.set_slippage(slippage.VolumeShareSlippage())


def handle_data(context, data):
    long_win = 60
    trailing_window = data.history(context.asset, 'price', long_win, '1d')
    if trailing_window.isnull().values.any():
        return
    short_ema = EMA(trailing_window.values, timeperiod=long_win/2)
    long_ema = EMA(trailing_window.values, timeperiod=long_win)

    buy = False
    sell = False

    if (short_ema[-1] > long_ema[-1]) and not context.invested:
        #order(context.asset, 100)
        order_target_percent(context.asset, 1)
        context.invested = True
        buy = True
    elif (short_ema[-1] < long_ema[-1]) and context.invested:
        #order(context.asset, -100)
        order_target_percent(context.asset, 0)
        context.invested = False
        sell = True

    record(AAPL=data.current(context.asset, "price"),
           short_ema=short_ema[-1],
           long_ema=long_ema[-1],
           buy=buy,
           sell=sell)


# Note: this function can be removed if running
# this algorithm on quantopian.com
def analyze(context=None, results=None):
    import matplotlib.pyplot as plt
    import logbook
    logbook.StderrHandler().push_application()
    log = logbook.Logger('Algorithm')

    fig = plt.figure()
    ax1 = fig.add_subplot(212)
    results.portfolio_value.plot(ax=ax1)
    ax1.set_ylabel('Portfolio value (USD)')

    ax2 = fig.add_subplot(211)
    ax2.set_ylabel('Price (USD)')

    # If data has been record()ed, then plot it.
    # Otherwise, log the fact that no data has been recorded.
    if 'AAPL' in results and 'short_ema' in results and 'long_ema' in results:
        results[['AAPL', 'short_ema', 'long_ema']].plot(ax=ax2)

        ax2.plot(
            results.index[results.buy],
            results.loc[results.buy, 'long_ema'],
            '^',
            markersize=10,
            color='m',
        )
        ax2.plot(
            results.index[results.sell],
            results.loc[results.sell, 'short_ema'],
            'v',
            markersize=10,
            color='k',
        )
        plt.legend(loc=0)
        plt.gcf().set_size_inches(18, 8)
    else:
        msg = 'AAPL, short_ema and long_ema data not captured using record().'
        ax2.annotate(msg, xy=(0.1, 0.5))
        log.info(msg)
    plt.gcf().set_size_inches(18, 8)
    plt.show() 

[2018-10-22 11:54:05.369524] INFO: zipline.finance.metrics.tracker: Simulated 755 trading days
first open: 2015-01-02 14:31:00+00:00
last close: 2017-12-29 21:00:00+00:00


AAPL  algo_volatility  algorithm_period_return  \
2015-01-02 21:00:00+00:00  109.330              NaN                 0.000000   
2015-01-05 21:00:00+00:00  106.250         0.001122                -0.000100   
2015-01-06 21:00:00+00:00  106.260         0.001517                -0.000009   
2015-01-07 21:00:00+00:00  107.750         0.107653                 0.013550   
2015-01-08 21:00:00+00:00  111.890         0.257318                 0.051224   
2015-01-09 21:00:00+00:00  112.010         0.237596                 0.052316   
2015-01-12 21:00:00+00:00  109.250         0.291640                 0.027200   
2015-01-13 21:00:00+00:00  110.220         0.271242                 0.036027   
2015-01-14 21:00:00+00:00  109.800         0.257539                 0.032105   
2015-01-15 21:00:00+00:00  106.820         0.243494                 0.032105   
2015-01-16 21:00:00+00:00  105.990         0.231529                 0.032105   
2015-01-20 21:00:00+00:00  108.720         0.221174                 0.032105   
2015-01-21 21:00:00+00:00  109.550         0.212097                 0.032105   
2015-01-22 21:00:00+00:00  112.400         0.204056                 0.032105   
2015-01-23 21:00:00+00:00  112.980         0.196866                 0.032105   
2015-01-26 21:00:00+00:00  113.100         0.190387                 0.032105   
2015-01-27 21:00:00+00:00  109.140         0.184510                 0.032105   
2015-01-28 21:00:00+00:00  115.310         0.179145                 0.032105   
2015-01-29 21:00:00+00:00  118.900         0.174223                 0.032105   
2015-01-30 21:00:00+00:00  117.160         0.169687                 0.032105   
2015-02-02 21:00:00+00:00  118.630         0.165487                 0.032105   
2015-02-03 21:00:00+00:00  118.650         0.161596                 0.032005   
2015-02-04 21:00:00+00:00  119.560         0.159204                 0.039922   
2015-02-05 21:00:00+00:00  119.940         0.155774                 0.043228   
2015-02-06 21:00:00+00:00  118.930         0.155917                 0.034441   
2015-02-09 21:00:00+00:00  119.720         0.153636                 0.041314   
2015-02-10 21:00:00+00:00  122.020         0.159974                 0.061324   
2015-02-11 21:00:00+00:00  124.880         0.169365                 0.086206   
2015-02-12 21:00:00+00:00  126.460         0.168725                 0.099952   
2015-02-13 21:00:00+00:00  127.080         0.165852                 0.105346   
...                            ...              ...                      ...   
2017-11-16 21:00:00+00:00  171.100         0.161431                 0.235399   
2017-11-17 21:00:00+00:00  170.150         0.161356                 0.228654   
2017-11-20 21:00:00+00:00  169.980         0.161247                 0.227447   
2017-11-21 21:00:00+00:00  173.140         0.161481                 0.249883   
2017-11-22 21:00:00+00:00  174.960         0.161477                 0.262805   
2017-11-24 18:00:00+00:00  174.970         0.161366                 0.262876   
2017-11-27 21:00:00+00:00  174.090         0.161286                 0.256628   
2017-11-28 21:00:00+00:00  173.070         0.161216                 0.249386   
2017-11-29 21:00:00+00:00  169.480         0.161564                 0.223897   
2017-11-30 21:00:00+00:00  171.850         0.161645                 0.240724   
2017-12-01 21:00:00+00:00  171.050         0.161561                 0.235044   
2017-12-04 21:00:00+00:00  169.800         0.161511                 0.226169   
2017-12-05 21:00:00+00:00  169.640         0.161403                 0.225033   
2017-12-06 21:00:00+00:00  169.010         0.161311                 0.220560   
2017-12-07 21:00:00+00:00  169.452         0.161207                 0.223699   
2017-12-08 21:00:00+00:00  169.370         0.161098                 0.223116   
2017-12-11 21:00:00+00:00  172.670         0.161363                 0.246546   
2017-12-12 21:00:00+00:00  171.700         0.161291                 0.239659   
2017-12-13 21:00:00+00:00  172

In [23]:
import pandas as pd
perf = pd.read_pickle('../data/strategy3.pickle')
print("Benchmark:{}, Our strategy: {}".format(perf['benchmark_period_return'][-1], perf['algorithm_period_return'][-1]))
perf[['benchmark_period_return','algorithm_period_return']]


Benchmark:0.379827139153, Our strategy: 0.222122499988


benchmark_period_return  algorithm_period_return
2015-01-02 21:00:00+00:00                -0.000535                 0.000000
2015-01-05 21:00:00+00:00                -0.018585                -0.000100
2015-01-06 21:00:00+00:00                -0.027829                -0.000009
2015-01-07 21:00:00+00:00                -0.015715                 0.013550
2015-01-08 21:00:00+00:00                 0.001752                 0.051224
2015-01-09 21:00:00+00:00                -0.006276                 0.052316
2015-01-12 21:00:00+00:00                -0.014060                 0.027200
2015-01-13 21:00:00+00:00                -0.016834                 0.036027
2015-01-14 21:00:00+00:00                -0.022769                 0.032105
2015-01-15 21:00:00+00:00                -0.031722                 0.032105
2015-01-16 21:00:00+00:00                -0.019023                 0.032105
2015-01-20 21:00:00+00:00                -0.016955                 0.032105
2015-01-21 21:00:00+00:00                -0.011968                 0.032105
2015-01-22 21:00:00+00:00                 0.002725                 0.032105
2015-01-23 21:00:00+00:00                -0.002773                 0.032105
2015-01-26 21:00:00+00:00                -0.000438                 0.032105
2015-01-27 21:00:00+00:00                -0.013622                 0.032105
2015-01-28 21:00:00+00:00                -0.026272                 0.032105
2015-01-29 21:00:00+00:00                -0.017270                 0.032105
2015-01-30 21:00:00+00:00                -0.029629                 0.032105
2015-02-02 21:00:00+00:00                -0.017612                 0.032105
2015-02-03 21:00:00+00:00                -0.003405                 0.032005
2015-02-04 21:00:00+00:00                -0.007200                 0.039922
2015-02-05 21:00:00+00:00                 0.002822                 0.043228
2015-02-06 21:00:00+00:00                 0.000049                 0.034441
2015-02-09 21:00:00+00:00                -0.004427                 0.041314
2015-02-10 21:00:00+00:00                 0.006179                 0.061324
2015-02-11 21:00:00+00:00                 0.006763                 0.086206
2015-02-12 21:00:00+00:00                 0.016445                 0.099952
2015-02-13 21:00:00+00:00                 0.020629                 0.105346
...                                            ...                      ...
2017-11-16 21:00:00+00:00                 0.330488                 0.235399
2017-11-17 21:00:00+00:00                 0.326578                 0.228654
2017-11-20 21:00:00+00:00                 0.328842                 0.227447
2017-11-21 21:00:00+00:00                 0.337536                 0.249883
2017-11-22 21:00:00+00:00                 0.336353                 0.262805
2017-11-24 18:00:00+00:00                 0.339440                 0.262876
2017-11-27 21:00:00+00:00                 0.338771                 0.256628
2017-11-28 21:00:00+00:00                 0.352353                 0.249386
2017-11-29 21:00:00+00:00                 0.351530                 0.223897
2017-11-30 21:00:00+00:00                 0.363362                 0.240724
2017-12-01 21:00:00+00:00                 0.360533                 0.235044
2017-12-04 21:00:00+00:00                 0.358886                 0.226169
2017-12-05 21:00:00+00:00                 0.353999                 0.225033
2017-12-06 21:00:00+00:00                 0.354256                 0.220560
2017-12-07 21:00:00+00:00                 0.358526                 0.223699
2017-12-08 21:00:00+00:00                 0.365934                 0.223116
2017-12-11 21:00:00+00:00                 0.370050                 0.246546
2017-12-12 21:00:00+00:00                 0.372468                 0.239659
2017-12-13 21:00:00+00:00                 0.372313                 0.243706
2017-12-14 21:00:00+00:00                 0.366706                 0.243351
2017-12-15 21:00:00+00:00                 0.378017                 0.255066